In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json
from collections import defaultdict

class WikipediaGraphScraper:
    def __init__(self):
        self.base_url = "https://vi.wikipedia.org/wiki/"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
        })
        self.visited = set()
        self.nodes = {}
        self.edges = []

    def get_page(self, title):
        """Lấy nội dung HTML của một trang Wikipedia"""
        try:
            url = self.base_url + title.replace(" ", "_")
            response = self.session.get(url, timeout=10)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return BeautifulSoup(response.content, 'html.parser')
        except Exception as e:
            print(f"Lỗi khi truy cập {title}: {e}")
        return None

    def extract_infobox(self, soup):
        """Trích xuất thông tin từ infobox"""
        infobox = {}
        infobox_table = soup.find('table', {'class': 'infobox'})

        if infobox_table:
            rows = infobox_table.find_all('tr')
            for row in rows:
                cells = row.find_all(['th', 'td'])
                if len(cells) >= 2:
                    key = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)
                    infobox[key] = value

        return infobox

    def extract_links(self, soup, title):
        """Trích xuất các liên kết từ trang Wikipedia"""
        links = []
        content = soup.find('div', {'id': 'mw-content-text'})

        if content:
            for link in content.find_all('a', href=True):
                href = link['href']
                # Chỉ lấy các link nội bộ Wikipedia
                if href.startswith('/wiki/'):
                    linked_title = href.replace('/wiki/', '').replace('_', ' ')
                    link_text = link.get_text(strip=True)

                    # Tránh các link đặc biệt
                    if not any(x in href for x in ['#', 'Special:', 'File:', 'Wikipedia:']):
                        links.append({
                            'target': linked_title,
                            'text': link_text,
                            'source': title
                        })

        return links

    def scrape_artist(self, artist_name, max_depth=2, current_depth=0):
        """Thu thập dữ liệu về một nghệ sĩ và các liên kết"""
        if artist_name in self.visited or current_depth > max_depth:
            return

        self.visited.add(artist_name)
        print(f"Đang xử lý: {artist_name} (Độ sâu: {current_depth})")

        soup = self.get_page(artist_name)
        if not soup:
            return

        # Trích xuất infobox
        infobox = self.extract_infobox(soup)

        # Tạo node
        self.nodes[artist_name] = {
            'type': 'artist',
            'infobox': infobox,
            'url': self.base_url + artist_name.replace(" ", "_")
        }

        # Trích xuất các liên kết
        links = self.extract_links(soup, artist_name)

        # Lọc các liên kết liên quan (album, bài hát, nghệ sĩ khác, v.v.)
        relevant_keywords = ['album', 'bài hát', 'ca sĩ', 'nhạc sĩ', 'nhóm nhạc',
                            'đơn ca', 'nhạc Hàn Quốc', 'K-pop', 'OST']

        for link in links[:30]:  # Giới hạn 30 liên kết per trang
            target = link['target']
            link_text = link['text']

            # Kiểm tra nếu là liên kết liên quan
            is_relevant = any(kw in link_text.lower() or kw in target.lower()
                             for kw in relevant_keywords)

            if is_relevant or current_depth < max_depth:
                # Tạo edge
                edge = {
                    'source': artist_name,
                    'target': target,
                    'relation': link_text,
                    'relation_type': 'related_to'
                }
                self.edges.append(edge)

                # Đệ quy thu thập dữ liệu
                if target not in self.visited:
                    self.scrape_artist(target, max_depth, current_depth + 1)

    def save_graph_data(self, filename='graph_data.json'):
        """Lưu dữ liệu graph"""
        data = {
            'nodes': self.nodes,
            'edges': self.edges,
            'statistics': {
                'total_nodes': len(self.nodes),
                'total_edges': len(self.edges)
            }
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"\nDữ liệu đã lưu vào {filename}")
        print(f"Tổng nodes: {len(self.nodes)}")
        print(f"Tổng edges: {len(self.edges)}")

    def print_node_info(self, node_name):
        """In thông tin chi tiết của một node"""
        if node_name in self.nodes:
            node = self.nodes[node_name]
            print(f"\n=== Node: {node_name} ===")
            print(f"URL: {node['url']}")
            print(f"\nInfobox:")
            for key, value in node['infobox'].items():
                print(f"  {key}: {value}")

            # In các liên kết
            print(f"\nCác liên kết từ node này:")
            related_edges = [e for e in self.edges if e['source'] == node_name]
            for edge in related_edges[:10]:
                print(f"  -> {edge['target']} ({edge['relation']})")
        else:
            print(f"Node '{node_name}' không được tìm thấy")


# Ví dụ sử dụng
if __name__ == "__main__":
    scraper = WikipediaGraphScraper()

    # Bắt đầu từ một ca sĩ/nhạc sĩ Hàn Quốc nổi tiếng
    start_artist = "BTS"  # Thay đổi theo ý muốn

    print(f"Bắt đầu thu thập dữ liệu từ: {start_artist}\n")
    scraper.scrape_artist(start_artist, max_depth=2)

    # In thông tin của node bắt đầu
    scraper.print_node_info(start_artist)

    # Lưu dữ liệu
    scraper.save_graph_data('korean_artists_graph.json')

Bắt đầu thu thập dữ liệu từ: BTS

Đang xử lý: BTS (Độ sâu: 0)
Đang xử lý: BTS (%C4%91%E1%BB%8Bnh h%C6%B0%E1%BB%9Bng) (Độ sâu: 1)
Đang xử lý: Tr%E1%BA%A1m thu ph%C3%A1t s%C3%B3ng di %C4%91%E1%BB%99ng (Độ sâu: 2)
Đang xử lý: X%C3%A2y d%E1%BB%B1ng d%E1%BB%B1 tr%E1%BB%AF (Độ sâu: 2)
Đang xử lý: S%C3%A2n bay M. R. %C5%A0tef%C3%A1nik (Độ sâu: 2)
Đang xử lý: %C4%90%E1%BA%B7c bi%E1%BB%87t:Ti%E1%BB%81n t%E1%BB%91/BTS (Độ sâu: 2)
Đang xử lý: %C4%90%E1%BA%B7c bi%E1%BB%87t:T%C3%ACm ki%E1%BA%BFm/intitle:BTS (Độ sâu: 2)
Đang xử lý: T%E1%BA%ADp tin:Disambig gray.svg (Độ sâu: 2)
Đang xử lý: T%E1%BA%ADp tin:BTS during a White House press conference May 31, 2022 (cropped).jpg (Độ sâu: 1)
Đang xử lý: Th%C3%A0nh vi%C3%AAn:Btspurplegalaxy (Độ sâu: 2)
Đang xử lý: Danh s%C3%A1ch tweet c%C3%B3 nhi%E1%BB%81u l%C6%B0%E1%BB%A3t th%C3%ADch nh%E1%BA%A5t (Độ sâu: 2)
Đang xử lý: %C4%90%E1%BA%B7c bi%E1%BB%87t:S%E1%BB%AD d%E1%BB%A5ng to%C3%A0n c%E1%BB%A5c/BTS during a White House press conference May 31, 2022 (cropped

In [8]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json
from collections import defaultdict, deque
import time

class WikipediaGraphScraper:
    def __init__(self):
        self.base_url = "https://vi.wikipedia.org/wiki/"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
        })
        self.visited = set()
        self.nodes = {}
        self.edges = []
        self.artist_keywords = ['ca sĩ', 'nhạc sĩ', 'diễn viên', 'nhóm nhạc', 'K-pop',
                               'album', 'bài hát', 'OST', 'giải thưởng']

    def get_page(self, title):
        """Lấy nội dung HTML của một trang Wikipedia"""
        try:
            url = self.base_url + title.replace(" ", "_")
            response = self.session.get(url, timeout=10)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return BeautifulSoup(response.content, 'html.parser')
        except Exception as e:
            print(f"Lỗi khi truy cập {title}: {e}")
        return None

    def extract_infobox(self, soup):
        """Trích xuất thông tin từ infobox"""
        infobox = {}
        infobox_table = soup.find('table', {'class': 'infobox'})

        if infobox_table:
            rows = infobox_table.find_all('tr')
            for row in rows:
                cells = row.find_all(['th', 'td'])
                if len(cells) >= 2:
                    key = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)
                    infobox[key] = value

        return infobox

    def extract_links(self, soup, title):
        """Trích xuất các liên kết từ trang Wikipedia"""
        links = []
        content = soup.find('div', {'id': 'mw-content-text'})

        if content:
            for link in content.find_all('a', href=True):
                href = link['href']
                if href.startswith('/wiki/'):
                    linked_title = href.replace('/wiki/', '').replace('_', ' ')
                    link_text = link.get_text(strip=True)

                    if not any(x in href for x in ['#', 'Special:', 'File:', 'Wikipedia:']):
                        links.append({
                            'target': linked_title,
                            'text': link_text,
                            'source': title
                        })

        return links

    def is_artist_related(self, text):
        """Kiểm tra xem một liên kết có liên quan đến ca sĩ/nhạc sĩ"""
        text_lower = text.lower()
        return any(kw in text_lower for kw in self.artist_keywords)

    def calculate_relevance_score(self, node_name, infobox, links):
        """Tính điểm liên quan của một node"""
        score = 0

        # Kiểm tra trong tên
        if any(kw in node_name.lower() for kw in ['ca sĩ', 'nhạc sĩ', 'K-pop']):
            score += 10

        # Kiểm tra trong infobox
        infobox_text = ' '.join(infobox.values()).lower()
        if 'ca sĩ' in infobox_text or 'nhạc sĩ' in infobox_text or 'K-pop' in infobox_text:
            score += 8

        # Đếm liên kết liên quan
        artist_links = sum(1 for link in links if self.is_artist_related(link['text']))
        score += min(artist_links, 10)

        return score

    # ============= CÁC THUẬT TOÁN DUYỆT =============

    def bfs_expansion(self, seed_artists, max_nodes=1000, max_links_per_node=15):
        """
        Thuật toán BFS (Breadth-First Search)
        - Duyệt theo tầng
        - Đảm bảo phát hiện tất cả các node gần với hạt giống
        - Phù hợp với mạng rộng, nông
        """
        print("\n" + "="*70)
        print("THUẬT TOÁN: BFS (Breadth-First Search)")
        print("="*70)

        queue = deque([(artist, 0) for artist in seed_artists])
        self.visited = set(seed_artists)
        level_stats = defaultdict(int)

        while queue and len(self.nodes) < max_nodes:
            current_node, depth = queue.popleft()
            level_stats[depth] += 1

            print(f"[BFS Tầng {depth}] Xử lý: {current_node}")

            soup = self.get_page(current_node)
            if not soup:
                continue

            infobox = self.extract_infobox(soup)
            self.nodes[current_node] = {
                'type': 'artist',
                'infobox': infobox,
                'url': self.base_url + current_node.replace(" ", "_"),
                'depth': depth
            }

            links = self.extract_links(soup, current_node)
            artist_links = [l for l in links if self.is_artist_related(l['text'])][:max_links_per_node]

            for link in artist_links:
                target = link['target']
                if target not in self.visited and len(self.nodes) < max_nodes:
                    self.visited.add(target)
                    queue.append((target, depth + 1))

                    self.edges.append({
                        'source': current_node,
                        'target': target,
                        'relation': link['text'],
                        'relation_type': 'related_to'
                    })

            time.sleep(0.5)

        print(f"\nThống kê BFS theo tầng: {dict(level_stats)}")
        return self.nodes, self.edges

    def dfs_expansion(self, seed_artists, max_nodes=1000, max_depth=3, max_links_per_node=10):
        """
        Thuật toán DFS (Depth-First Search)
        - Duyệt theo chiều sâu
        - Phát hiện các chuỗi liên kết dài
        - Phù hợp với mạng có cấu trúc phân nhánh
        """
        print("\n" + "="*70)
        print("THUẬT TOÁN: DFS (Depth-First Search)")
        print("="*70)

        self.visited = set()
        depth_stats = defaultdict(int)

        def dfs_helper(node, depth):
            if len(self.nodes) >= max_nodes or depth > max_depth or node in self.visited:
                return

            self.visited.add(node)
            depth_stats[depth] += 1

            print(f"[DFS Độ sâu {depth}] Xử lý: {node}")

            soup = self.get_page(node)
            if not soup:
                return

            infobox = self.extract_infobox(soup)
            self.nodes[node] = {
                'type': 'artist',
                'infobox': infobox,
                'url': self.base_url + node.replace(" ", "_"),
                'depth': depth
            }

            links = self.extract_links(soup, node)
            artist_links = [l for l in links if self.is_artist_related(l['text'])][:max_links_per_node]

            for link in artist_links:
                target = link['target']
                self.edges.append({
                    'source': node,
                    'target': target,
                    'relation': link['text'],
                    'relation_type': 'related_to'
                })
                dfs_helper(target, depth + 1)

            time.sleep(0.3)

        for seed in seed_artists:
            dfs_helper(seed, 0)

        print(f"\nThống kê DFS theo độ sâu: {dict(depth_stats)}")
        return self.nodes, self.edges

    def priority_expansion(self, seed_artists, max_nodes=1000, max_links_per_node=15):
        """
        Thuật toán Priority-based Expansion
        - Duyệt ưu tiên theo điểm liên quan
        - Sử dụng heuristic để ưu tiên các node quan trọng
        - Phù hợp với mạng có độ dày khác nhau
        """
        print("\n" + "="*70)
        print("THUẬT TOÁN: Priority-based Expansion")
        print("="*70)

        from heapq import heappush, heappop

        priority_queue = []
        self.visited = set()

        # Khởi tạo với hạt giống
        for seed in seed_artists:
            heappush(priority_queue, (-100, seed, 0))  # Ưu tiên cao nhất

        stats = {
            'high_priority': 0,
            'medium_priority': 0,
            'low_priority': 0
        }

        while priority_queue and len(self.nodes) < max_nodes:
            priority, current_node, depth = heappop(priority_queue)
            priority = -priority

            if current_node in self.visited:
                continue

            self.visited.add(current_node)

            print(f"[Priority {priority}] Xử lý: {current_node}")

            soup = self.get_page(current_node)
            if not soup:
                continue

            infobox = self.extract_infobox(soup)
            links = self.extract_links(soup, current_node)

            # Tính điểm liên quan
            relevance = self.calculate_relevance_score(current_node, infobox, links)

            self.nodes[current_node] = {
                'type': 'artist',
                'infobox': infobox,
                'url': self.base_url + current_node.replace(" ", "_"),
                'depth': depth,
                'relevance_score': relevance
            }

            artist_links = [l for l in links if self.is_artist_related(l['text'])][:max_links_per_node]

            for link in artist_links:
                target = link['target']
                if target not in self.visited and len(self.nodes) < max_nodes:
                    # Tính điểm cho node tiếp theo
                    next_relevance = max(0, relevance - 2)  # Giảm dần

                    if next_relevance > 15:
                        stats['high_priority'] += 1
                    elif next_relevance > 8:
                        stats['medium_priority'] += 1
                    else:
                        stats['low_priority'] += 1

                    heappush(priority_queue, (-next_relevance, target, depth + 1))

                    self.edges.append({
                        'source': current_node,
                        'target': target,
                        'relation': link['text'],
                        'relation_type': 'related_to',
                        'relevance_score': next_relevance
                    })

            time.sleep(0.5)

        print(f"\nThống kê Priority: {stats}")
        return self.nodes, self.edges

    def save_graph_data(self, filename='graph_data.json'):
        """Lưu dữ liệu graph"""
        data = {
            'nodes': self.nodes,
            'edges': self.edges,
            'statistics': {
                'total_nodes': len(self.nodes),
                'total_edges': len(self.edges)
            }
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"\nDữ liệu đã lưu vào {filename}")
        print(f"Tổng nodes: {len(self.nodes)}")
        print(f"Tổng edges: {len(self.edges)}")

    def print_summary(self):
        """In tóm tắt mạng lưới"""
        print("\n" + "="*70)
        print("TÓM TẮT MẠNG LƯỚI")
        print("="*70)
        print(f"Tổng nodes: {len(self.nodes)}")
        print(f"Tổng edges: {len(self.edges)}")
        print(f"Mật độ: {len(self.edges) / max(1, len(self.nodes)):.2f}")

        # Node có liên kết nhiều nhất
        edge_count = defaultdict(int)
        for edge in self.edges:
            edge_count[edge['source']] += 1

        if edge_count:
            top_nodes = sorted(edge_count.items(), key=lambda x: x[1], reverse=True)[:5]
            print("\nTop 5 node có liên kết nhiều nhất:")
            for node, count in top_nodes:
                print(f"  - {node}: {count} liên kết")


# Tập hạt giống
SEED_ARTISTS = [
    "BTS", "BLACKPINK", "TWICE", "IU", "EXO"
]


# Ví dụ sử dụng
if __name__ == "__main__":
    print("="*70)
    print("HỆ THỐNG DUYỆT TẬP HẠT GIỐNG - CA SĨ/NHẠC Sĩ HÀN QUỐC")
    print("="*70)
    print(f"\nTập hạt giống: {SEED_ARTISTS}")
    print(f"Số lượng hạt giống: {len(SEED_ARTISTS)}\n")

    # Chọn thuật toán
    print("Chọn thuật toán duyệt:")
    print("1. BFS - Breadth-First Search")
    print("2. DFS - Depth-First Search")
    print("3. Priority-based Expansion")

    choice = input("\nNhập lựa chọn (1/2/3): ").strip()

    scraper = WikipediaGraphScraper()

    if choice == "1":
        scraper.bfs_expansion(SEED_ARTISTS, max_nodes=100, max_links_per_node=15)
    elif choice == "2":
        scraper.dfs_expansion(SEED_ARTISTS, max_nodes=100, max_depth=3, max_links_per_node=10)
    elif choice == "3":
        scraper.priority_expansion(SEED_ARTISTS, max_nodes=100, max_links_per_node=15)
    else:
        print("Lựa chọn không hợp lệ, sử dụng BFS mặc định")
        scraper.bfs_expansion(SEED_ARTISTS, max_nodes=100)

    scraper.print_summary()
    scraper.save_graph_data(f'korean_artists_graph_{choice}.json')

HỆ THỐNG DUYỆT TẬP HẠT GIỐNG - CA SĨ/NHẠC Sĩ HÀN QUỐC

Tập hạt giống: ['BTS', 'BLACKPINK', 'TWICE', 'IU', 'EXO']
Số lượng hạt giống: 5

Chọn thuật toán duyệt:
1. BFS - Breadth-First Search
2. DFS - Depth-First Search
3. Priority-based Expansion

Nhập lựa chọn (1/2/3): 2

THUẬT TOÁN: DFS (Depth-First Search)
[DFS Độ sâu 0] Xử lý: BTS
[DFS Độ sâu 1] Xử lý: Nh%C3%B3m nh%E1%BA%A1c nam
[DFS Độ sâu 2] Xử lý: Nh%C3%B3m nh%E1%BA%A1c n%E1%BB%AF
[DFS Độ sâu 3] Xử lý: Ban nh%E1%BA%A1c
[DFS Độ sâu 3] Xử lý: AKB48 Group
[DFS Độ sâu 3] Xử lý: Gaon Album Chart
[DFS Độ sâu 3] Xử lý: Danh s%C3%A1ch nh%C3%B3m nh%E1%BA%A1c n%E1%BB%AF H%C3%A0n Qu%E1%BB%91c
[DFS Độ sâu 3] Xử lý: Album ch%E1%BB%A7 %C4%91%E1%BB%81
[DFS Độ sâu 3] Xử lý: Nh%C3%B3m nh%E1%BA%A1c
[DFS Độ sâu 2] Xử lý: Si%C3%AAu ban nh%E1%BA%A1c
[DFS Độ sâu 2] Xử lý: Danh s%C3%A1ch album b%C3%A1n ch%E1%BA%A1y nh%E1%BA%A5t t%E1%BA%A1i H%C3%A0n Qu%E1%BB%91c
[DFS Độ sâu 3] Xử lý: Danh s%C3%A1ch album %C4%91%E1%BA%A1t ch%E1%BB%A9ng nh%E1%BA%ADn t%E1%B

HỆ THỐNG DUYỆT TẬP HẠT GIỐNG - CA SĨ/NHẠC Sĩ HÀN QUỐC

Tập hạt giống: ['BTS', 'BLACKPINK', 'TWICE', 'IU', 'EXO']
Số lượng hạt giống: 5

Chọn thuật toán duyệt:
1. BFS - Breadth-First Search
2. DFS - Depth-First Search
3. Priority-based Expansion

Nhập lựa chọn (1/2/3): 1

THUẬT TOÁN: BFS (Breadth-First Search)
[BFS Tầng 0] Xử lý: BTS
[BFS Tầng 0] Xử lý: BLACKPINK
[BFS Tầng 0] Xử lý: TWICE
[BFS Tầng 0] Xử lý: IU
[BFS Tầng 0] Xử lý: EXO
[BFS Tầng 1] Xử lý: Nh%C3%B3m nh%E1%BA%A1c nam
[BFS Tầng 1] Xử lý: Danh s%C3%A1ch album b%C3%A1n ch%E1%BA%A1y nh%E1%BA%A5t t%E1%BA%A1i H%C3%A0n Qu%E1%BB%91c
[BFS Tầng 1] Xử lý: Danh s%C3%A1ch gi%E1%BA%A3i th%C6%B0%E1%BB%9Fng v%C3%A0 %C4%91%E1%BB%81 c%E1%BB%AD c%E1%BB%A7a BTS
[BFS Tầng 1] Xử lý: %C4%90%C4%A9a %C4%91%C6%A1n
[BFS Tầng 1] Xử lý: %C4%90%C4%A9a m%E1%BB%9F r%E1%BB%99ng
[BFS Tầng 1] Xử lý: Gaon Album Chart
[BFS Tầng 1] Xử lý: Oricon Albums Chart
[BFS Tầng 1] Xử lý: %C4%90%C6%A1n v%E1%BB%8B album t%C6%B0%C6%A1ng %C4%91%C6%B0%C6%A1ng
[BFS Tầng 1] Xử

In [18]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import json
from collections import defaultdict, deque
import time
import sys

# Đảm bảo output UTF-8 trên Windows
if sys.platform == 'win32':
    import io
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

class WikipediaBFScraper:
    def __init__(self):
        self.base_url = "https://vi.wikipedia.org/wiki/"
        self.session = requests.Session( )
        self.session.headers.update({
            'User-Agent': 'ManusAI/1.0 (Academic K-pop Graph Scraper)'
        })
        self.nodes = {}
        self.edges = []

        # Từ khóa để xác định một trang có liên quan đến chủ đề hay không
        self.topic_keywords = ['ca sĩ', 'nhạc sĩ', 'diễn viên', 'nhóm nhạc', 'k-pop',
                               'album', 'bài hát', 'ost', 'giải thưởng', 'công ty giải trí']

        # Các mẫu để xác định loại quan hệ giữa các node
        self.relation_patterns = {
            'MEMBER_OF': ['thành viên', 'tham gia nhóm'],
            'COLLABORATED_WITH': ['hợp tác', 'cộng tác', 'featuring', 'ft.', 'ft'],
            'PART_OF_ALBUM': ['album', 'bài hát trong', 'đĩa đơn từ'],
            'IN_DRAMA_OR_OST': ['phim', 'tập phim', 'ost', 'nhạc phim'],
            'MANAGED_BY': ['công ty quản lý', 'hãng đĩa', 'agency', 'quản lý bởi'],
            'PRODUCED': ['sản xuất', 'nhà sản xuất', 'producer', 'sáng tác'],
            'WON_AWARD': ['giải thưởng', 'đoạt giải', 'nhận giải', 'chiến thắng'],
            'SUB_UNIT_OF': ['nhóm nhỏ', 'dự án nhóm'],
        }

    def get_page_soup(self, title):
        """Tải và phân tích HTML của một trang Wikipedia."""
        try:
            url = self.base_url + title.replace(" ", "_")
            response = self.session.get(url, timeout=10)
            response.raise_for_status() # Báo lỗi nếu request không thành công
            # Sử dụng BeautifulSoup để phân tích HTML
            return BeautifulSoup(response.content, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"  ✗ Lỗi mạng khi truy cập '{title}': {e}")
        return None

    def extract_infobox(self, soup):
        """Trích xuất dữ liệu từ bảng infobox bên phải trang."""
        infobox_data = {}
        infobox_table = soup.find('table', class_='infobox')
        if infobox_table:
            for row in infobox_table.find_all('tr'):
                header = row.find('th')
                data = row.find('td')
                if header and data:
                    key = header.get_text(strip=True)
                    value = data.get_text(strip=True)
                    infobox_data[key] = value
        return infobox_data

    def classify_relation(self, context_text):
        """Phân loại mối quan hệ dựa trên văn bản xung quanh liên kết."""
        context_lower = context_text.lower()
        for rel_type, keywords in self.relation_patterns.items():
            if any(keyword in context_lower for keyword in keywords):
                return rel_type
        return 'RELATED_TO' # Trả về loại mặc định nếu không tìm thấy

    def is_relevant_page(self, title, soup):
        """Kiểm tra xem trang có liên quan đến chủ đề ca sĩ/nhạc sĩ Hàn Quốc không."""
        text_content = soup.get_text().lower()
        # Một trang được coi là liên quan nếu có ít nhất 2 từ khóa chủ đề
        relevance_score = sum(1 for keyword in self.topic_keywords if keyword in text_content)
        return relevance_score >= 2

    def scrape_with_bfs(self, seed_artists, max_nodes=1000):
        """
        Sử dụng thuật toán BFS (Breadth-First Search) để xây dựng mạng lưới.
        - Bắt đầu từ các node hạt giống.
        - Duyệt qua từng "tầng" các node liên quan.
        - Dừng lại khi đạt đủ số lượng node.
        """
        print("\n" + "="*70)
        print("Bắt đầu thu thập dữ liệu bằng thuật toán BFS (Tìm kiếm theo chiều rộng)")
        print("="*70)

        queue = deque([(artist, 0) for artist in seed_artists]) # (tên_node, độ_sâu)
        visited = set(seed_artists)

        while queue and len(self.nodes) < max_nodes:
            current_title, depth = queue.popleft()

            print(f"[Tầng {depth}] Đang xử lý: {current_title} ({len(self.nodes) + 1}/{max_nodes})")

            soup = self.get_page_soup(current_title)
            if not soup:
                continue

            # Chỉ xử lý các trang có liên quan
            if not self.is_relevant_page(current_title, soup):
                print(f"  - Bỏ qua trang không liên quan: {current_title}")
                continue

            infobox = self.extract_infobox(soup)

            # Lưu thông tin node
            self.nodes[current_title] = {
                'label': 'Artist/Group', # Cần cải tiến để phân loại rõ hơn
                'infobox': infobox,
                'url': self.base_url + current_title.replace(" ", "_"),
                'depth': depth
            }

            # Tìm tất cả các liên kết hợp lệ trong phần nội dung
            content_div = soup.find('div', id='mw-content-text')
            if not content_div:
                continue

            for link in content_div.find_all('a', href=True):
                href = link['href']
                if href.startswith('/wiki/') and ':' not in href and '#' not in href:
                    target_title = href.replace('/wiki/', '').replace('_', ' ')

                    if target_title not in visited and len(self.nodes) < max_nodes:
                        visited.add(target_title)
                        queue.append((target_title, depth + 1))

                        # Tạo cạnh với thông tin quan hệ
                        relation_type = self.classify_relation(link.get_text())
                        self.edges.append({
                            'source': current_title,
                            'target': target_title,
                            'type': relation_type,
                            'text': link.get_text(strip=True)
                        })

            time.sleep(0.2) # Tạm dừng một chút để tránh gửi quá nhiều yêu cầu

        print("\n✓ Thu thập dữ liệu hoàn tất!")

    def save_data(self, filename='korean_artists_graph_bfs.json'):
        """Lưu dữ liệu node và cạnh vào file JSON."""
        graph_data = {
            'nodes': self.nodes,
            'edges': self.edges,
            'stats': {
                'node_count': len(self.nodes),
                'edge_count': len(self.edges),
            }
        }
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(graph_data, f, ensure_ascii=False, indent=2)
        print(f"✓ Dữ liệu đã được lưu vào tệp: {filename}")

# --- PHẦN THỰC THI CHÍNH ---

if __name__ == "__main__":
    # Danh sách 10 node hạt giống bạn đã chọn
    SEED_ARTISTS = [
        "BTS", "Blackpink", "Big Bang (nhóm nhạc)", "Girls' Generation", "EXO",
        "Red Velvet (nhóm nhạc)", "NCT (nhóm nhạc)", "Psy", "IU (ca sĩ)", "Cha Eun-woo"
    ]

    # Số lượng node tối đa cần thu thập (theo yêu cầu là tối thiểu 1000)
    MAX_NODES_TO_COLLECT = 1000

    print("="*70)
    print("CHƯƠNG TRÌNH XÂY DỰNG MẠNG LƯỚI NGHỆ SĨ HÀN QUỐC")
    print(f"Bắt đầu từ {len(SEED_ARTISTS)} node hạt giống, mục tiêu {MAX_NODES_TO_COLLECT} node.")

    scraper = WikipediaBFScraper()
    scraper.scrape_with_bfs(SEED_ARTISTS, max_nodes=MAX_NODES_TO_COLLECT)
    scraper.save_data()

    # In ra một vài thống kê cơ bản
    print("\n--- THỐNG KÊ MẠNG LƯỚI ---")
    print(f"Tổng số node thu thập được: {len(scraper.nodes)}")
    print(f"Tổng số cạnh (mối quan hệ) tìm thấy: {len(scraper.edges)}")



CHƯƠNG TRÌNH XÂY DỰNG MẠNG LƯỚI NGHỆ SĨ HÀN QUỐC
Bắt đầu từ 10 node hạt giống, mục tiêu 1000 node.

Bắt đầu thu thập dữ liệu bằng thuật toán BFS (Tìm kiếm theo chiều rộng)
[Tầng 0] Đang xử lý: BTS (1/1000)
[Tầng 0] Đang xử lý: Blackpink (2/1000)
[Tầng 0] Đang xử lý: Big Bang (nhóm nhạc) (3/1000)
[Tầng 0] Đang xử lý: Girls' Generation (4/1000)
[Tầng 0] Đang xử lý: EXO (5/1000)
[Tầng 0] Đang xử lý: Red Velvet (nhóm nhạc) (6/1000)
[Tầng 0] Đang xử lý: NCT (nhóm nhạc) (7/1000)
[Tầng 0] Đang xử lý: Psy (8/1000)
[Tầng 0] Đang xử lý: IU (ca sĩ) (9/1000)
[Tầng 0] Đang xử lý: Cha Eun-woo (10/1000)
[Tầng 1] Đang xử lý: BTS (%C4%91%E1%BB%8Bnh h%C6%B0%E1%BB%9Bng) (11/1000)
  - Bỏ qua trang không liên quan: BTS (%C4%91%E1%BB%8Bnh h%C6%B0%E1%BB%9Bng)
[Tầng 1] Đang xử lý: Nh%C3%A0 Tr%E1%BA%AFng (11/1000)
  - Bỏ qua trang không liên quan: Nh%C3%A0 Tr%E1%BA%AFng
[Tầng 1] Đang xử lý: Seoul (11/1000)
[Tầng 1] Đang xử lý: H%C3%A0n Qu%E1%BB%91c (12/1000)
[Tầng 1] Đang xử lý: K-pop (13/1000)
[Tầng 1] Đang x